In [266]:
import tulip as tlp
from tulip.interfaces import omega as omega_int
from tulip import transys, abstract, spec, synth
from visualization import graph_builder as gb
import networkx as nx
from tulip.transys import machines
from GR1_defaults import settings

from tulip import dumpsmach
import pickle

In [267]:
path = 'var_inc/'

In [268]:
! rm -r var_inc

In [269]:
mkdir var_inc

In [270]:
# Variables
env_vars = {'is_true':'boolean',
            'is_true2': 'boolean'}
sys_vars = {'num':(0,1)}

In [271]:
increment = False

In [272]:
# Initialization
env_init = {'is_true', 'is_true2'}
# env_init = set()
if increment:
    sys_init = {'num=0'}
else:
    sys_init = {'num=1'}

In [273]:
# Safety
env_safe = {'(!is_true) -> X(is_true)',
            '(is_true) -> !X(is_true)',
            'X is_true -> X is_true2' # does not count initial state
}
if increment:
    sys_safe = {'(num=0) -> (X(num)=num+1)',
                '(num=1) -> (X(num)=1)'}
else:
    sys_safe = {'(num=1) -> (X(num)=num-1)',
                '(num=0) -> (X(num)=0)'}



In [274]:
# Progress
env_prog = set()
if increment:
    sys_prog = {'num=1'}
else:
    sys_prog = {'num=0'}

In [275]:
specs = settings.set_specs(env_vars, sys_vars, env_init, sys_init,
                            env_safe, sys_safe, env_prog, sys_prog)

In [276]:
print(specs.pretty())

ENVIRONMENT VARIABLES:
	is_true	boolean
	is_true2	boolean

SYSTEM VARIABLES:
	num	(0, 1)

FORMULA:
ASSUMPTION:
    INITIAL
	  (is_true2)
	& (is_true)
    SAFETY
	  []((is_true) -> !X(is_true))
	& []((!is_true) -> X(is_true))
	& []((is_true) -> (is_true <=> is_true2))
GUARANTEE:
    INITIAL
	  (num=1)
    SAFETY
	  []((num=0) -> (X(num)=0))
	& []((num=1) -> (X(num)=num-1))
    LIVENESS
	  []<>(num=0)



In [277]:
spec = tlp.synth._spec_plus_sys(specs, None, None, False, False)

In [278]:
# Automaton class found in omega/omega/symbolic/temporal.py
aut = omega_int._grspec_to_automaton(spec)

Error  : expr IMP LPAREN expr LE . LexToken(GE,'>',1,24)


Exception: Syntax error at ">"
remaining input:
is_true2 )


In [ ]:
# Synthesizing system controller
ctrl = tlp.synth.synthesize(specs)
assert ctrl is not None, 'unrealizable'

removed 0 nodes from 5 total


In [ ]:
 # Graphing
filename = path + "graph"
attributes = ['color', 'shape']

In [ ]:
# Making a graph of the asynchronous GR(1) game with deadends.
g0 = gb.game_graph(aut, env='env', sys='sys', remove_deadends=False, qinit=aut.qinit)
h0 = gb._game_format_nx(g0, attributes)
pd0 = nx.drawing.nx_pydot.to_pydot(h0)
pd0.write_pdf(path + 'game.pdf')
# with open(filename, "wb") as file:
#     pickle.dump(g0, file)